In [24]:
# If alignment file other format, rewrite this function
def extractIDs(fastaAlignment):
    with open(fastaAlignment) as f:
        IDs = [line.split("|")[1] for line in f.readlines() if line.startswith(">")]
    return IDs

In [27]:
from apiFunctions import uniprotDownload
import pandas as pd

# Extract Ids from Fasta
ids = extractIDs("exampleAlignment.fasta")

# Infromation for API
fileName="proteins.tab"
query="id:"+"+OR+id:".join(ids)
format="tab"

# You can find possible columns on this page https://www.uniprot.org/help/uniprotkb_column_names
columns="id,entry name,genes,organism,comment(PTM),feature(SIGNAL),3d,database(EMBL)"

# Download File
uniprotDownload(fileName,query=query, format=format, columns=columns)

# Show with Pandas
pd.read_csv(fileName, sep="\t")

,Entry,Entry name,Gene names,Organism,Post-translational modification,Signal peptide,3D,Cross-reference (EMBL)
0,Q59641,PPIA_PSEAE,ppiA cypH PA3227,Pseudomonas aeruginosa (strain ATCC 15692 / DS...,NaN,"SIGNAL 1..20; /evidence=""ECO:0000255""",NaN,AE004091;X84050;
1,P42693,PPIA_ACIAD,rotA ppiA ACIAD3647,Acinetobacter baylyi (strain ATCC 33305 / BD41...,NaN,"SIGNAL 1..20; /evidence=""ECO:0000255""",NaN,X74839;CR543861;
2,D0ZFY3,D0ZFY3_EDWTE,ppiA ETAE_3250,Edwardsiella tarda (strain EIB202),NaN,"SIGNAL 1..24; /evidence=""ECO:0000256|RuleBase...",NaN,CP001135;
3,Q7N9E8,Q7N9E8_PHOLL,ppiA plu0391,Photorhabdus laumondii subsp. laumondii (strai...,NaN,"SIGNAL 1..19; /evidence=""ECO:0000256|RuleBase...",NaN,BX571860;
4,W6MAX4,W6MAX4_9GAMM,ppiA BN873_770017,Candidatus Competibacter denitrificans Run_A_D11,NaN,"SIGNAL 1..22; /evidence=""ECO:0000256|RuleBase...",NaN,CBTJ020000088;
...,...,...,...,...,...,...,...,...
120,A0A1A9AZZ1,A0A1A9AZZ1_PLESH,ppiA C7R88_14995 NCTC10363_02400,Plesiomonas shigelloides (Aeromonas shigelloides),NaN,"SIGNAL 1..21; /evidence=""ECO:0000256|RuleBase...",NaN,CP027852;UGTC01000001;
121,A0A0S2SI89,A0A0S2SI89_9GAMM,ppiA ATO46_17905 WL1483_2003,Aeromonas schubertii,NaN,"SIGNAL 1..17; /evidence=""ECO:0000256|RuleBase...",NaN,CP013067;LPUO01000018;
122,A0A142BCF4,A0A142BCF4_9GAMM,ppiA EZMO1_2333,Endozoicomonas montiporae CL-33,NaN,"SIGNAL 1..26; /evidence=""ECO:0000256|RuleBase...",NaN,CP013251;
123,Q0VQ95,Q0VQ95_ALCBS,ppiA ABO_1205,Alcanivorax borkumensis (strain ATCC 700651 / ...,NaN,"SIGNAL 1..17; /evidence=""ECO:0000256|RuleBase...",NaN,AM286690;


In [26]:
for id in id

A0A510XAS8
A0A4Y3JFH8
A0A0R4J726
P42693
A0A380UEM9
A0A380UC84
A0A1X7AP38
A0A379DM05
A0A485AFZ9
A0A2X1VAN9
A0A3B0JAH8
D2U3K4
B4EZ63
A0A379FKV6
Q7N9E8
D3VEA5
A0A1N6MQT8
A0A447RWL8
A0A378A0D2
A0A4P8YG64
I2B4D5
A0A5J6VYW8
D2A938
A0A2X3EDZ7
A0A2X2T901
A0A4U2UDM3
A0A085JP37
W0HNX6
A0A221T4P4
B2VK29
A0A1E7Z141
A0A506V8F3
A0A0U5LJT7
D4GCL5
D0ZFY3
A0A389M9I0
A0A377PQ82
A0A5B8I1D1
A0A5J5FTJ9
E9CMV2
A0A4U9U0A5
A0A0T9KRE4
Q8CZJ4
A0A084A040
Q48LN3
A0A5E6T1S2
A0A446AYX8
A0A5E6QRG9
Q1I6R5
A0A5E6SWH6
I4L4W4
A0A2C8ERN2
A0A5E6TGQ2
A0A5E7KMF5
A0A2X2EE17
A0A379KA38
S6ATG3
A0A098FNZ9
A0A078LSA2
Q59641
A0A024HLJ9
A0A127MXJ7
Q488X1
A0A509DW67
F3BHB9
A0A0P9FXN8
A0A098G264
A0A447KNK0
A0A379W1P3
A0A377LRD7
A0A447PW19
A0A376TWL1
A0A377DLA4
A0A377C5I8
A0A2X1JSG5
A0A2N7QRI8
H8W6H0
A0A5E7XUU0
W6MAX4
Q0VQ95
A0A2R8CHL8
A0A381EF53
A0A380MYZ6
Q5QWT2
A0A0P9JWD4
A0A510XUL3
A0A5H2XU43
A0A510U0Y2
A0A2S2DZX2
A0A5H3A543
A0A0P7XU76
Q488Q5
Q8EAT0
D4ZBC9
A0A252EQQ8
A0A379ZFC9
A0A485FJ98
A0A1C9WBM0
A0A509DYM4
W2UDZ0
A4SQM4
A0A0S